In [1]:
import os

In [2]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.ML_Project.constants import *
from src.ML_Project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])         

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.ML_Project import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]] 
        y_test = test_data[[self.config.target_column]]

        # define the model
        lr_model = ElasticNet(
            alpha= self.config.alpha,
            l1_ratio= self.config.l1_ratio,
            random_state= 42
        )
        
        # train the model
        lr_model.fit(X_train, y_train)

        # save the model
        joblib.dump(lr_model, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-03-23 18:24:12,407: INFO: common: Yaml file:config\config.yaml loaded successfully]
[2025-03-23 18:24:12,413: INFO: common: Yaml file:params.yaml loaded successfully]
[2025-03-23 18:24:12,420: INFO: common: Yaml file:schema.yaml loaded successfully]
[2025-03-23 18:24:12,423: INFO: common: created directory at: artifacts]
[2025-03-23 18:24:12,426: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\karth\anaconda3\envs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.242e+14, tolerance: 1.187e+11
  model = cd_fast.enet_coordinate_descent(
